In [ ]:
import numpy as np
import pandas as pd  #manejo de datos
import statsmodels.api as sm #para mco
import statsmodels.formula.api as smf
import matplotlib.pylab as plt #para gráficos

from wooldridge import *


In [ ]:
#Para introducir datos, lectura del modelo...

from wooldridge import *
datos=dataWoo("hprice1") #para datos de wooldridge
y = datos['lprice'] #variable endógena 
X = datos[['llotsize', 'lsqrft', 'bdrms']] #Variables exógenas


mco = sm.OLS(y,sm.add_constant(X)).fit()
mco.summary()

#Para excel o csv, y utilizando pandas

datos = pd.read_csv('Theil.csv', delimiter=";")  #los datos deben estar en la misma carpeta que el archivo de jupyter

datos= pd.read_excel ('stock.xlsx') #Leer datos de archivo excel

#Para aplicar logaritmos a las variables
datos["lY"] = np.log(datos["Y"])
datos["lX1"] = np.log(datos["X1"]) #Estamos creando nuevos datos
datos["lX2"] = np.log(datos["X2"])

y=datos["lY"] 
X=datos[["lX1","lX2"]]  #Definimos estos datos nuevos como variables

In [ ]:
# MULTICOLINEALIDAD

#COMO ENCONTRARLA
mco.condition_number # Número de Condición, >20 moderado, >30 preocupante,,,,relación exógenas y cte, multi.aprox.no esencial

import statsmodels.stats.outliers_influence as oi

vifs=[oi.variance_inflation_factor(X.values, i) for i in range(X.shape[1])]  # si son menores que 10, no hay multicolinealidad
vifs            #no relaciona exógenas con la cte

#TIPOS
#  sistemática... correlación lineal entre exógenas
#  errática... problema numérico... aumento de la muestra
#  no esencial... relación lineal exógenas con cte... hay que centrar las variables
#  esencial... relación lineal entre las variables exógenas

In [ ]:
# CORRELACIÓN LINEAL
corr_matrix=np.corrcoef(X.T)
print(corr_matrix)
import statsmodels.graphics.api as smg
smg.plot_corr(corr_matrix, xnames=["","", "", "","",""])
plt.show()

In [ ]:
# HETEROSCEDASTICIDAD
import statsmodels.stats.api as sms
BP=sms.het_breuschpagan(mco.resid,mco.model.exog)#['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
W=sms.het_white(mco.resid,mco.model.exog)
sms.het_goldfeldquandt(mco.resid, mco.model.exog) #['F statistic', 'p-value']

labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
print(dict(zip(labels, bp_test)))
print(dict(zip(labels, white_test)))

# También podemos intuir este problema con gráficos de residuos y de dispersión
# Para arreglar la heteroscedasticidad realizamos el test de glejser para saber donde está el problema
#una vez realizado el test, aplicamos mínimos cuadrados poderados, scrip de basedatos

In [ ]:
# AUTOCORRELACIÓN

from statsmodels.stats.stattools import durbin_watson
dw=durbin_watson(mco.resid)  #para realizar el test

#si tenemos problemas de autocorrelación...
#DW aprox 0, entonces correlacion positiva
#DW aprox 2, entonces no hay correlacion
#DW aprox 4, entonces correacion negativa
# Para arreglarlo...

rho= 1 - dw/2

mco_autocorr=sm.GLSAR(y, sm.add_constant(X), rho=rho).iterative_fit(maxiter=1000)  #Modificación Prais-Winsten
mco_autocorr.summary()

In [ ]:
# UN TEST DE CHOW
#realizamos este test para analizar si existen diferencias entre los resultados de dos muestras

datos1987=datos10[datos10["year"]==1987]  #modelo para 1987
Y1987=datos1987["infmort"]
X1987=datos1987[["lpcinc", "lpopul", "lphysic"]]
mco1987=sm.OLS(Y1987, sm.add_constant(X1987)).fit()
SCR1987=mco1987.ssr


datos1990=datos10[datos10["year"]==1990]  #modelo para 1990
Y1990=datos1990["infmort"]
X1990=datos1990[["lpcinc", "lpopul", "lphysic"]]
mco1990=sm.OLS(Y1990, sm.add_constant(X1990)).fit()
SCR1990=mco1990.ssr


n=len(datos10)
k=3 #numero variables

Fexp=(n-2*k-2)/(k+1) * (SCR - (SCR1987+SCR1990))/(SCR1987+SCR1990)
Fexp

from scipy import stats
Fteo= stats.f.ppf(1-0.05, k+1, n-2*k-1)

print(Fexp, Fteo) #se rechaza que 1990 y 1987 son iguales si Fexp>Fteo
print(datos10)

In [ ]:
#Para realizar tests de hipotesis R*B=r
mco7.f_test(([0,1,1],1))
#Al no poder rechazar la hip. nula porque el p-valor es más grande que 0.05. Damos por valido que 𝛽1+𝛽2=1.
#dentro de [] ponemos los coeficientes de los beta, y a la derecha ponemos el r
#Si la suma de los beta es 1, un incremento conjunto de los X de 1% aumenta un 1% la Y

In [ ]:
import numpy as np
A= np.matrix([[0.114,-0.0005,-0.0002],[-0.005,0.00002,-0.00002],[-0.0002,-0.00002,0.00003]],float)
B= np.matrix([[443.818],[66792.89],[53125.748]])
CoefB= A.dot(B)
#print(A)
#print(B)
from scipy import stats
print(CoefB)